<a href="https://colab.research.google.com/github/Pedfortunato/ADVANCED-ML/blob/main/MACHINELEARNING(SISTEMASDERECOMENDACAO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sistemas de Recomendação**


Essa prática irá apresentar passo a passo a implementação de um sistema de recomendação baseado em filtros colaborativos, onde será gerada a matriz de ratings, e serão calculadas as similaridades entre os usuários

1) Para iniciar vamos importar a biblioteca pandas, do Python, que possui estruturas de dados muito úteis para manipular as informações. Iremos utilizar os datasets movies.csv e Rating.csv

In [ ]:
import pandas as pd
dataFile='ratings.csv'
data=pd.read_csv(dataFile,sep=",",header=0,names=["userId","movieId","rating","timestamp"])
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
movieFile='movies.csv'
movies=pd.read_csv(movieFile,sep=",",header=0,names=["movieId","title","genres"])
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


2) vamos criar um método bem simples, que irá receber como parâmetro o ID de um filme e irá retornar o seu título e os gêneros associados a ele.

In [ ]:
def movieMeta(movieId):
    title = movies.at[movieId,"title"]
    genre = movies.at[movieId,"genres"]
    return title, genre

movieMeta(2)

('Grumpier Old Men (1995)', 'Comedy|Romance')

3) Agora que temos uma função que retorna os dados básicos do filme, para vermos o que foi recomendado, precisamos criar uma para retornar os filmes favoritos de um usuário.

In [ ]:
def faveMovies(user,N):
    #userRatings recebe a lista de ratings do usuário passado como parâmetro
    userRatings = data[data["userId"]==user]
    #SortedRatings recebe os N primeiros itens dessa lista ordenada de forma decrescente
    sortedRatings = pd.DataFrame.sort_values(userRatings,['rating'],ascending=[0])[:N]
    #o método movieMeta é aplicado nos N itens mais relevantes, para que a coluna title
    #seja criada no nosso dataframe sortedRatings, para que possamos visualizar.
    sortedRatings["title"] = sortedRatings["movieId"].apply(movieMeta)
    return sortedRatings


faveMovies(1,60)

,userId,movieId,rating,timestamp,title
4,1,1172,4.0,1260759205,"(Gridlock'd (1997), Crime)"
13,1,2105,4.0,1260759139,"(Besieged (a.k.a. L' Assedio) (1998), Drama)"
12,1,1953,4.0,1260759191,"(At First Sight (1999), Drama)"
8,1,1339,3.5,1260759125,"(Red Corner (1997), Crime|Thriller)"
19,1,3671,3.0,1260759117,"(War of the Roses, The (1989), Comedy|Drama)"
1,1,1029,3.0,1260759179,"(Highlander (1986), Action|Adventure|Fantasy)"
2,1,1061,3.0,1260759182,"(Hype! (1996), Documentary)"
14,1,2150,3.0,1260759194,"(General's Daughter, The (1999), Crime|Drama|M..."
17,1,2455,2.5,1260759113,"(Pokémon: The First Movie (1998), Adventure|An..."
0,1,31,2.5,1260759144,"(Twelve Monkeys (a.k.a. 12 Monkeys) (1995), My..."


In [ ]:
data.shape

(100004, 4)

4) userPerMOVIE recebe quantos usuários viram cada filme.

In [ ]:
usersPerMOVIE = data.movieId.value_counts()
usersPerMOVIE.head(10)

,count
movieId,
356,341
296,324
318,311
593,304
260,291
480,274
2571,259
1,247
527,244


In [ ]:
usersPerMOVIE.shape

(9066,)

5) fazemos o mesmo para verificar quantos usuários viram um certo filme

In [ ]:
MOVIESsPerUser = data.userId.value_counts()
MOVIESsPerUser.head()

,count
userId,
547,2391
564,1868
624,1735
15,1700
73,1610


In [ ]:
MOVIESsPerUser.shape

(671,)

6) Agora vamos obter com a célula abaixo os filmes vistos por mais de 10 usuários e na próxima os usuários que viram mais de 10 filmes, como uma forma de redução simples de dimensionalidade.


In [ ]:
data = data[data["movieId"].isin(usersPerMOVIE[usersPerMOVIE>10].index)]
data = data[data["userId"].isin(MOVIESsPerUser[MOVIESsPerUser>10].index)]

7) Feito isso, vamos utilizar o método pivot_table do pandas dataframe, para gerar nossa matriz de ratings, onde os valores serão os ratings, os índices, ou linhas, serão os usuários e as colunas o ID dos filmes, com base nos nossos dados de ratings, lidos do arquivo rating.csv no começo do notebook.


In [ ]:
userItemRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['userId'], columns=['movieId'])

userItemRatingMatrix.head()

movieId,1,2,3,4,5,6,7,9,10,11,...,117176,119145,122882,122886,122892,122900,122904,134130,134853,139385
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
userItemRatingMatrix.shape

(671, 2083)

In [ ]:
user1 = 1
user2 = 20

user1Ratings = userItemRatingMatrix.transpose()[user1]
user1Ratings.head(100)

,1
movieId,
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
...,...
177,NaN
180,NaN
181,NaN


In [ ]:
user2Ratings = userItemRatingMatrix.transpose()[user2]

8) Agora precisamos definir uma forma de similaridade entre os usuários ou produtos. Iremos trabalhar apenas com os usuários.

Para isso foi utilizada a distância de hamming, existente na biblioteca scipy

In [ ]:
from scipy.spatial.distance import hamming
hamming(user1Ratings,user2Ratings)
import numpy as np

def distance(user1,user2):
        try:
            #recupera os ratings dos dois usuários e depois é aplicado o método hamming, para calcular
            #a distância, que será retornada.
            user1Ratings = userItemRatingMatrix.transpose()[user1]
            user2Ratings = userItemRatingMatrix.transpose()[user2]
            distance = hamming(user1Ratings,user2Ratings)
        except:
            distance = np.NaN
        return distance

distance(6,5)



0.99951992318771

9)agora iremos filtrar o usuário 6 e calcular a distância entre ele e todos os demais usuários


In [ ]:
user = 6
allUsers = pd.DataFrame(userItemRatingMatrix.index)
allUsers = allUsers[allUsers.userId!=user]
allUsers.head(10)

,userId
0,1
1,2
2,3
3,4
4,5
6,7
7,8
8,9
9,10
10,11


10) as distâncias são então calculadas aplicando a função de distância para todos os usuários em relação ao usuário 6.Isso é feito com a expressão lambda, que ira pegar a lista de todos os userId do nosso dataframe allusers e aplicar a função distance em relação ao nosso usuário 6. tudo isso com uma simples linha de código.

In [ ]:
allUsers["distance"] = allUsers["userId"].apply(lambda x: distance(user,x))
allUsers.head()

,userId,distance
0,1,1.00000
1,2,1.00000
2,3,1.00000
3,4,1.00000
4,5,0.99952


11)Agora podemos selecionar os K vizinhos mais próximos ordenando esse resultado das distâncias e pegando os K primeiros usuários resultantes dessa ordenação.

In [ ]:
K = 10
KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["userId"][:K]
KnearestUsers

,userId
72,73
467,468
29,30
252,253
104,105
118,119
653,654
212,213
546,547
211,212


12)Agora vamos criar um método que aplica essa seleção dos usuários diferentes do que desejamos fazer as recomendações, cálcula as similaridades e ordena o resultado, retornando apenas os mais próximos.

Esse método é o apresentado abaixo, chamado de nearestNeighbors, que recebe como parâmetro o usuário e o número de vizinhos.

In [ ]:
def nearestNeighbors(user,K=10):
    allUsers = pd.DataFrame(userItemRatingMatrix.index)
    allUsers = allUsers[allUsers.userId!=user]
    allUsers["distance"] = allUsers["userId"].apply(lambda x: distance(user,x))
    KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["userId"][:K]
    return KnearestUsers

KnearestUsers = nearestNeighbors(user)

KnearestUsers


,userId
72,73
467,468
29,30
252,253
104,105
118,119
653,654
212,213
546,547
211,212


13) vamos então com o código abaixo, filtrar a matriz de ratings, apenas com os usuários correspondentes aos vizinhos mais próximos.

In [ ]:
NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
NNRatings

movieId,1,2,3,4,5,6,7,9,10,11,...,117176,119145,122882,122886,122892,122900,122904,134130,134853,139385
userId,,,,,,,,,,,,,,,,,,,,,
30,4.0,2.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,5.0,2.5,NaN,NaN,NaN,4.5,NaN,NaN,3.0,NaN,...,4.0,3.5,5.0,4.5,NaN,3.0,4.5,4.5,NaN,4.0
105,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.5,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,2.0,3.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,3.0,3.0,NaN,NaN,2.5,NaN,NaN,NaN,3.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213,3.0,3.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,4.0,4.0,NaN,NaN,NaN,NaN
253,4.5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
468,4.0,2.0,2.5,NaN,NaN,3.5,NaN,NaN,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
547,3.5,NaN,NaN,NaN,NaN,2.5,2.0,NaN,NaN,3.0,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.5,4.0,3.5


14) Cálculamos então e média dos ratings dos vizinhos mais próximos, para determinar os ratings não observados para o usuário que desejamos.

In [ ]:
avgRating = NNRatings.apply(np.nanmean).dropna()
avgRating.head()

/usr/local/lib/python3.10/dist-packages/pandas/core/apply.py:981: RuntimeWarning: Mean of empty slice
  results[i] = self.func(v, *self.args, **self.kwargs)


,0
movieId,
1,3.777778
2,2.812500
3,2.500000
5,3.250000
6,3.750000


15) Agora precisamos filtrar os filmes que o usuário já assistiu, pois eles não devem entrar nas recomendações. Isso é feito com o filtro simples abaixo, que determina o índice dos filmes que o usuário possui rating observado.

In [ ]:
moviesAlreadyWatched = userItemRatingMatrix.transpose()[user].dropna().index
moviesAlreadyWatched

Index([ 111,  158,  173,  293,  596,  903, 1204, 1250, 1259, 1276, 1285, 1358,
       1639, 1687, 1747, 1876, 1909, 2001, 2019, 2072, 2174, 2502, 2528, 2529,
       2571, 2657, 2692, 2723, 2761, 2890, 3052, 3114, 3300, 3751, 4641, 4975,
       5952, 7090, 7153, 7361, 8368, 8636, 8784, 8874],
      dtype='int64', name='movieId')

16)É calculada a média dos filmes sem rating observado.

In [ ]:
avgRating = avgRating[~avgRating.index.isin(moviesAlreadyWatched)]


17) Então é feito um teste com os 3 filmes com melhor recomendações, baseado na ordenação das médias de ratings dos vizinhos mais próximos.

In [ ]:
N=3
topNMOVIEIDs = avgRating.sort_values(ascending=False).index[:N]

In [ ]:
pd.Series(topNMOVIEIDs).apply(movieMeta)

,movieId
0,"(Stargate (1994), Action|Adventure|Sci-Fi)"
1,"(Castle Freak (1995), Horror)"
2,"(Outside Providence (1999), Comedy)"


18) Tendo tudo isso, podemos encapsular em um método que executa todas essas etapas e retorna as recomendações, chamado de topN.

In [ ]:
def topN(user,N=3):
    KnearestUsers = nearestNeighbors(user)
    NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
    avgRating = NNRatings.apply(np.nanmean).dropna()
    moviesAlreadyWatched = userItemRatingMatrix.transpose()[user].dropna().index
    avgRating = avgRating[~avgRating.index.isin(moviesAlreadyWatched)]
    ratingPredictedValue = avgRating.sort_values(ascending=False)
    #print(ratingPredictedValue.head(N))
    topNMOVIEIDs = avgRating.sort_values(ascending=False).index[:N]
    recommendation = pd.DataFrame(topNMOVIEIDs, columns=['movieId'])
    # Ensure the movie IDs exist in the movies DataFrame before applying movieMeta
    recommendation = recommendation[recommendation['movieId'].isin(movies.index)]
    recommendation["title"] = recommendation["movieId"].apply(movieMeta)
    recommendation["Prediction"] = ratingPredictedValue.loc[recommendation["movieId"]].values
    return recommendation


19) Veja o exemplo abaixo, onde são recomendados 70 filmes, com base na média das distâncias, que funcionam como a similaridade

In [ ]:
topN(6,70)

/usr/local/lib/python3.10/dist-packages/pandas/core/apply.py:981: RuntimeWarning: Mean of empty slice
  results[i] = self.func(v, *self.args, **self.kwargs)


,movieId,title,Prediction
0,282,"(Stargate (1994), Action|Adventure|Sci-Fi)",5.0
1,194,"(Castle Freak (1995), Horror)",5.0
2,2269,"(Outside Providence (1999), Comedy)",5.0
3,215,"(Gordy (1995), Children|Comedy|Fantasy)",5.0
4,1797,"(One True Thing (1998), Drama)",5.0
...,...,...,...
64,322,"(Four Weddings and a Funeral (1994), Comedy|Ro...",4.5
66,7132,"(Blackadder Back & Forth (1999), Comedy)",4.5
67,2718,"(Captain Horatio Hornblower R.N. (1951), Actio...",4.5
68,1213,"(Hollow Reed (1996), Drama)",4.5


In [ ]:
topN(8,80)

/usr/local/lib/python3.10/dist-packages/pandas/core/apply.py:981: RuntimeWarning: Mean of empty slice
  results[i] = self.func(v, *self.args, **self.kwargs)


,movieId,title,Prediction
0,913,"(Hustler White (1996), Romance)",5.000000
2,2078,"(Open Your Eyes (Abre los ojos) (1997), Drama|...",5.000000
3,2132,"(It Came from Outer Space (1953), Sci-Fi)",5.000000
4,3089,"(Cell, The (2000), Drama|Horror|Thriller)",5.000000
5,6242,"(Anderson Tapes, The (1971), Crime|Drama|Thril...",5.000000
6,3071,"(Pumpkinhead (1988), Horror)",5.000000
7,1209,"(Grosse Pointe Blank (1997), Comedy|Crime|Roma...",5.000000
8,2301,"(Excalibur (1981), Adventure|Fantasy)",5.000000
9,3007,"(Fun and Fancy Free (1947), Animation|Children...",5.000000
10,1354,(Midnight in the Garden of Good and Evil (1997...,5.000000
